In [1]:
import sqlite3

con = sqlite3.connect("../scripts/2020-12-03.db", isolation_level=None)
con.execute("PRAGMA journal_mode=WAL;")
cursor = con.cursor()

In [2]:
cursor.execute("SELECT COUNT(*) FROM mace_output;")
row_count = cursor.fetchone()[0]

In [3]:
import re
from tqdm.notebook import tqdm

cursor.execute("SELECT output FROM mace_output")
err = 0
total = 0
for row in tqdm(range(row_count)):
    total += 1
    output = cursor.fetchone()[0]
    search_result = re.search(r".*INPUT(.*)end of input.*", output, re.DOTALL)
    input_lines = search_result.groups()[0]
    x = set()
    for equation in re.finditer(r"(\d+) \* (\d+) = (\d+)\.", input_lines):
        i, j, k = map(int, equation.groups())
        ind = 13 * i + j
        if ind in x:
            err += 1
            break
        else:
            x.add(ind)

In [4]:
print(err, total)

0 308220


In [5]:
cursor.execute("ALTER TABLE mace_output ADD COLUMN number_of_cells INTEGER")

OperationalError: duplicate column name: number_of_cells

In [6]:
cursor.execute("SELECT rowid, output FROM mace_output")
insert_cursor = con.cursor()
for row in tqdm(range(row_count)):
    data = cursor.fetchone()
    output = data[1]
    row_id = data[0]
    search_result = re.search(r".*INPUT(.*)end of input.*", output, re.DOTALL)
    input_lines = search_result.groups()[0]
    eq_num = 0
    for equation in re.finditer(r"(\d+) \* (\d+) = (\d+)\.", input_lines):
        eq_num += 1
    insert_cursor.execute(
        "UPDATE mace_output SET number_of_cells = ? WHERE rowid = ?",
        (eq_num, row_id)
    )

In [9]:
cursor.execute("""
SELECT
    number_of_cells,
    AVG(model_exists),
    SUM(model_exists),
    COUNT(*)
FROM mace_output
GROUP BY number_of_cells
ORDER BY number_of_cells
""")
cursor.fetchall()

[(1, 1.0, 1880, 1880),
 (2, 1.0, 1820, 1820),
 (3, 1.0, 1580, 1580),
 (4, 1.0, 2220, 2220),
 (5, 1.0, 1720, 1720),
 (6, 0.967391304347826, 1780, 1840),
 (7, 0.9295774647887324, 1320, 1420),
 (8, 0.946236559139785, 1760, 1860),
 (9, 0.8765432098765432, 1420, 1620),
 (10, 0.7191011235955056, 1280, 1780),
 (11, 0.39, 780, 2000),
 (12, 0.17708333333333334, 340, 1920),
 (13, 0.02727272727272727, 60, 2200),
 (14, 0.0, 0, 1780),
 (15, 0.0, 0, 1720),
 (16, 0.0, 0, 1940),
 (17, 0.0, 0, 2000),
 (18, 0.0, 0, 1800),
 (19, 0.0, 0, 1520),
 (20, 0.0, 0, 1920),
 (21, 0.0, 0, 2160),
 (22, 0.0, 0, 1700),
 (23, 0.0, 0, 1680),
 (24, 0.0, 0, 1660),
 (25, 0.0, 0, 1960),
 (26, 0.0, 0, 2020),
 (27, 0.0, 0, 1820),
 (28, 0.0, 0, 2080),
 (29, 0.0, 0, 1880),
 (30, 0.0, 0, 1660),
 (31, 0.0, 0, 2280),
 (32, 0.0, 0, 1740),
 (33, 0.0, 0, 2260),
 (34, 0.0, 0, 1700),
 (35, 0.0, 0, 1620),
 (36, 0.0, 0, 1900),
 (37, 0.0, 0, 1700),
 (38, 0.0, 0, 2300),
 (39, 0.0, 0, 1840),
 (40, 0.0, 0, 2000),
 (41, 0.0, 0, 1960),
 (42, 0